In [39]:
# # # 更新.pyc文件
# import compileall
# compileall.compile_dir(r'/Users/yantingting/PycharmProjects/modeling')

In [12]:
import sys
import os
import time
import numpy as np
import pandas as pd
from imp import reload
import pickle
import matplotlib
matplotlib.use('MacOSX')
import matplotlib.pyplot as plt
sys.path.append('/Users/yantingting/PycharmProjects/modeling')
import plot_tools as pt
import plotting as pl
import metrics as mt
from metrics import *
import misc_utils as mu
import database_conncet as data_con
import summary_statistics as ss
import feature_selection as fs
from feature_selection import *
from data_io_utils import *
import data_processing as dp
import generate_report as gr
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
pd.options.display.latex.repr = True
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

fs_obj = fs.FeatureSelection()
pf = mt.Performance()
bw = mt.BinWoe()

In [2]:
input_path = '/Users/yantingting/Seafile/风控/模型/14 巴西/Portocred/data/'
result_path = os.path.join('/Users/yantingting/Seafile/风控/模型/14 巴西/Portocred/','模型_LR(fpd2)/') 
if not os.path.exists(result_path):
    os.mkdir(result_path)

# 读取数据&核验数据

In [22]:
all_df = load_data_from_pickle(input_path,'df_var_all.pkl') 
# all_df = load_data_from_pickle('/Users/yantingting/Seafile/风控/模型/14 巴西/Portocred/模型_LR(1)/','all_all_original.pkl') 
all_df.index = all_df.loan_id
all_df.shape
all_df.rename(columns = {'date':'applied_at', 'fpd_2':'label'}, inplace=True)
all_df = all_df.loc[all_df.label.isin([0,1])]
all_df.shape
eda = ss.eda(all_df , result_path)
all_df.fillna(-1,inplace = True)
y_col = ['label']
useless_col = ['loan_id','applied_at','sample_flag','occupation_Capitalista rend aplic / aluguel','fpd_1','fpd_3']
x_col = list(set(all_df.columns)-set(useless_col)-set(y_col))
len(x_col)
save_data_to_pickle(all_df,result_path,'all_original.pkl')
for cols in x_col:
    try:
        all_df[cols] = all_df[cols].astype(float)
    except:
        pass

(445, 446)

(445, 446)

439

In [13]:
#字典
var_dict = pd.read_csv(os.path.join(input_path,'dict_all.csv'))
var_dict['指标类型'] = var_dict['数据源']
var_dict.shape
var_dict.head()

(440, 5)

,数据源,指标英文,指标中文,数据类型,指标类型
0,bank,serasa_score_y,serasa_score_y,int64,bank
1,bank,boa_vista_(bvs)_score_y,boa_vista_(bvs)_score_y,int64,bank
2,bank,internal_score_y,internal_score_y,int64,bank
3,bank,age,age,int64,bank
4,bank,social_status,social_status,int64,bank


# 划分数据集

In [24]:
#划分训练和验证
train_df = all_df[all_df.sample_flag == 0]
test_df = all_df[all_df.sample_flag == 1]
train_df.shape
train_df.label.value_counts(dropna = False)
test_df.shape
test_df.label.value_counts(dropna = False)
X_train = train_df[x_col]
X_test = test_df[x_col]
y_train = train_df.label
y_test = test_df.label
#检查数据量是否一致
X_train.shape[0] == y_train.shape[0]
X_test.shape[0] == y_test.shape[0]

(333, 446)

0    227
1    106
Name: label, dtype: int64

(112, 446)

0    77
1    35
Name: label, dtype: int64

True

True

# Univariate Chart

In [25]:
# feature_list = x_col
# time1 = time.time()
# wrong_list= []
# for index,value in enumerate(feature_list):
#     print(index,value)
#     try:
#         pt.univariate_chart(df = all_df.copy(), col = value , target ='label' , n = 5,
#                              special_attribute = [-1, np.nan],
#                              dftrain=train_df.copy(), dftest=test_df.copy(),
#                              draw_all=True, draw_train_test=True,result_path = result_path)
#     except Exception as e:
#         wrong_list.append(value)
#         print(e)
# wrong_list
# time2 = time.time()
# print('run_time: ', time2-time1)

# 计算XGB importance

In [26]:
model = XGBClassifier()
model.fit(X_train, y_train)
## prediction
y_pred = model.predict(X_test);
predictions = [round(value) for value in y_pred]
## evaluation
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

features_in_model = x_col
feature_importance = model.feature_importances_
var_importance = pd.DataFrame(columns=["指标英文", 'importance'])
var_importance['指标英文'] = features_in_model
var_importance['importance'] = feature_importance
var_importance.loc[:, 'importance_rank'] = var_importance.importance.rank(ascending=False)
var_importance.sort_values(by = 'importance',ascending = False).head()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Accuracy: 66.07%


,指标英文,importance,importance_rank
348,Spotify,0.035326,1.0
118,Banco Inter,0.029776,2.0
361,Santander,0.028142,3.0
78,low_freq_app,0.022549,4.0
28,phone_charging_status,0.022282,5.0


# overall_ranking

In [27]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
args_dict = {
    'random_forest': {
        'grid_search': False,
        'param': None
    },
    'xgboost': {
        'grid_search': False,
        'param': None
    }
}
methods = [
    'random_forest',
    #'lasso',
    #'xgboost'
]

time1 = time.time()
## Train 分箱,为了得到分箱的woe值
X_cat_train, X_transformed_train, woe_iv_df, rebin_spec, ranking_result = fs_obj.overall_ranking(X_train, y_train,
                                                                                           var_dict, args_dict,
                                                                                           methods, num_max_bins=5)

woe_iv_df.to_excel(os.path.join(result_path,'woe_iv_df.xlsx'))
ranking_result.to_excel(os.path.join(result_path,'ranking_result.xlsx'))

rebin_spec = mu.convert_rebin_spec2XGB_rebin_spec(rebin_spec)
rebin_spec_bin_adjusted = {k: v for k, v in rebin_spec.items()}

bin_obj = mt.BinWoe()
X_cat_train = bin_obj.convert_to_category(X_train, var_dict, rebin_spec_bin_adjusted)
X_cat_test = bin_obj.convert_to_category(X_test, var_dict, rebin_spec_bin_adjusted)

"""按训练集和验证集检查变量的稳定性和有效性"""
## train
train_df['appmon'] = '0_train'
test_df['appmon'] = '1_test'

all_cat = pd.concat([X_cat_train,X_cat_test])
app_data = pd.concat([train_df[['label','appmon']],test_df[['label','appmon']]])
X_cat_with_y_appmon_all = pd.merge(all_cat,app_data[['label','appmon']] ,left_index=True,right_index=True)
X_cat_with_y_appmon_all.shape

var_dist_badRate_by_time_all = ss.get_badRate_and_dist_by_time(X_cat_with_y_appmon_all,list([i for i in all_cat.columns if i != 'loan_id']),'appmon','label')
var_dist_badRate_by_time_all.to_excel(os.path.join(result_path, 'var_dist_badRate_by_sample.xlsx'))

# PSI
var_psi = pf.variable_psi(X_cat_train, X_cat_test, var_dict)
var_psi.loc[:, 'psi_rank'] = var_psi.PSI.rank(ascending=False)
var_psi.to_excel(os.path.join(result_path, 'var_PSI.xlsx'))

# 汇总各项指标
# var_psi.drop(['数据源','指标中文'],axis = 1,inplace=True)
ranking_result_all = ranking_result.merge(var_importance, on='指标英文', how='left').merge(var_psi, on='指标英文', how='left')
ranking_result_all.to_excel(os.path.join(result_path,'ranking_result_all.xlsx'))

time2 = time.time()
print('run_time: ', time2-time1)


/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


'按训练集和验证集检查变量的稳定性和有效性'

/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(445, 441)

run_time:  45.32127618789673


# 变量筛选

In [28]:
time1 = time.time()
train = train_df.drop(useless_col,axis = 1).drop('appmon',axis = 1)
test = test_df.drop(useless_col,axis = 1).drop('appmon',axis = 1)
f_rmv = feature_remove(train,test, ranking_result_all, result_path, psi = 0.1, iv = 0.02, imp = 0, corr = 0.65, slope = 'TRUE')

try:
    train = train.drop(['slope'], axis=1)
    test = test.drop(['slope'], axis=1)
except:
    pass

feature_used = list(set(x_col) - set(f_rmv))
len(feature_used)
X_train = train_df[feature_used]
X_test = test_df[feature_used]
time2 = time.time()
print('run_time: ', time2-time1)

变量总个数439
PSI筛掉的个数37
IV筛掉的个数365
xgb重要性筛掉的个数0
相关性特征筛掉的个数325
univariate斜率特征筛掉的个数92
总共筛掉的个数416
剩余变量个数23


23

run_time:  18.31294012069702


# 变量分箱并替换WOE

In [29]:
time1 = time.time()

## Train 分箱
X_cat_train, X_transformed, woe_iv_df, rebin_spec, ranking_result = fs_obj.overall_ranking(X_train, y_train,
                                                                                           var_dict, args_dict,
                                                                                           methods, num_max_bins=5)

save_data_to_pickle(X_cat_train,result_path,'X_cat_train.pkl')
save_data_to_pickle(woe_iv_df,result_path,'woe_iv_df.pkl')
save_data_to_pickle(rebin_spec,result_path,'rebin_spec.pkl')
save_data_to_pickle(X_transformed,result_path,'X_transformed.pkl')
save_data_to_pickle(ranking_result,result_path,'ranking_result.pkl')

rebin_spec = mu.convert_rebin_spec2XGB_rebin_spec(rebin_spec)
rebin_spec_bin_adjusted = {k:v for k,v in rebin_spec.items()}

bin_obj = mt.BinWoe()

X_cat_train = bin_obj.convert_to_category(X_train, var_dict, rebin_spec_bin_adjusted)
woe_iv_df_coarse = bin_obj.calculate_woe_all(X_cat_train, y_train, var_dict, rebin_spec_bin_adjusted)
X_transformed_train= bin_obj.transform_x_all(X_cat_train, woe_iv_df_coarse)
save_data_to_pickle(woe_iv_df_coarse,result_path,'woe_iv_df_coarse.pkl')
woe_iv_df_coarse.to_excel(os.path.join(result_path,'woe_iv_df_coarse.xlsx'))

X_cat_test = bin_obj.convert_to_category(X_test, var_dict, rebin_spec_bin_adjusted)
X_transformed_test= bin_obj.transform_x_all(X_cat_test, woe_iv_df_coarse)

time2 = time.time()
print('run_time: ', time2-time1)

run_time:  2.2096829414367676


# 训练模型

## 初跑模型

In [30]:
X_transformed_train['intercept'] = [1]*X_transformed_train.shape[0]
LR = sm.Logit(y_train,X_transformed_train).fit()
print(LR.summary())
pvalues = LR.pvalues.to_dict()

Optimization terminated successfully.
         Current function value: 0.510320
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  333
Model:                          Logit   Df Residuals:                      309
Method:                           MLE   Df Model:                           23
Date:                Thu, 19 Mar 2020   Pseudo R-squ.:                  0.1843
Time:                        01:46:49   Log-Likelihood:                -169.94
converged:                       True   LL-Null:                       -208.32
Covariance Type:            nonrobust   LLR p-value:                 1.049e-07
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
screen_(2016, 1080)                       0.6886      

## stepwise

In [31]:
# ### 有些变量不显著，需要逐步剔除
pvals = LR.pvalues
multi_analysis = feature_used
len(multi_analysis)
varLargeP = {k: v for k,v in pvals.items() if v >= 0.4}
varLargeP = sorted(varLargeP.items(), key=lambda d:d[1], reverse = True)

while(len(varLargeP) > 0 and len(multi_analysis) > 0):
    # 每次迭代中，剔除不显著的变量，直到
    # (1) 剩余所有变量均显著
    # (2) 没有特征可选
    varMaxP = varLargeP[0][0]
    print(varMaxP)
    if varMaxP == 'intercept':
        print('the intercept is not significant!')
        break
    multi_analysis.remove(varMaxP)
    y = y_train
    X = X_transformed_train[multi_analysis]
    X['intercept'] = [1] * X.shape[0]
    LR_final = sm.Logit(y, X).fit()
    pvals = LR_final.pvalues
    pvals = pvals.to_dict()
    varLargeP = {k: v for k, v in pvals.items() if v >= 0.4}
    varLargeP = sorted(varLargeP.items(), key=lambda d: d[1], reverse=True)

print(LR_final.summary())
frame = LR_final.params.to_frame(name = 'coef').merge(LR_final.pvalues.to_frame(name = 'P>|z|'),
                                              left_index = True,right_index = True,how = 'left')
frame.to_excel(os.path.join(result_path,'model_result.xlsx'))

23

email1_hotmail
Optimization terminated successfully.
         Current function value: 0.510321
         Iterations 7
education_degree_7
Optimization terminated successfully.
         Current function value: 0.510470
         Iterations 7
model_motorola one
Optimization terminated successfully.
         Current function value: 0.510834
         Iterations 7
education_degree_8
Optimization terminated successfully.
         Current function value: 0.511394
         Iterations 7
state_name_MA
Optimization terminated successfully.
         Current function value: 0.512160
         Iterations 7
email1_gmail
Optimization terminated successfully.
         Current function value: 0.513231
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  333
Model:                          Logit   Df Residuals:                      315
Method:                           MLE   Df Model:   

/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## VIF

In [32]:
X = np.matrix(X_transformed_train[feature_used])
VIF_list = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
max_VIF = max(VIF_list)
print(max_VIF)

1.1442048875232216


## COR

In [33]:
X_transformed_train[feature_used].corr().to_excel(os.path.join(result_path,'corr.xlsx'))

## 保存模型

In [34]:
saveModel =open(os.path.join(result_path,'LR_Model_Normal.pkl'),'wb')
pickle.dump(LR_final,saveModel)
saveModel.close()

##加载模型
# modelFile =open(result_path+'LR_Model_Normal.pkl','rb')
# LR = pickle.load(modelFile)
# modelFile.close()

# 结果

In [35]:
X_transformed_test['intercept']=[1]*X_transformed_test.shape[0]
y_train_pred = LR_final.predict(X_transformed_train[feature_used + ['intercept']])
y_train_pred.shape
y_test_pred = LR_final.predict(X_transformed_test[feature_used + ['intercept']])
y_test_pred.shape
# ### 打分&KS
train_pred = train_df.copy()
train_pred['y_pred'] = y_train_pred

test_pred = test_df.copy()
test_pred['y_pred'] = y_test_pred
data_scored_train, train_proba_ks, train_proba_ks_20, train_score_ks, train_score_ks_20, data_scored_test, test_proba_ks, test_proba_ks_20, test_score_ks, test_score_ks_20 = pf.data_score_KS(train_pred, test_pred, 'y_pred',n1 = 5,n2 = 10)

writer = pd.ExcelWriter(os.path.join(result_path,'ks.xlsx'))
train_score_ks.to_excel(writer,sheet_name = 'train_score_ks')
test_score_ks.to_excel(writer, sheet_name = 'test_score_ks') 
writer.close()

def auc_acc_table(df):
    from sklearn.metrics import roc_auc_score
    y = df.label.values
    y_pred = df.y_pred
    ## AUC
    auc = roc_auc_score(y, y_pred)
    print("auc: %.2f" % auc)
    ## Accuracy
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y, predictions)
    print("Accuracy: %.4f%%" % (accuracy * 100.0))
    return auc, accuracy

auc_train, acc_train = auc_acc_table(train_pred)
auc_test, acc_test = auc_acc_table(test_pred)
print('success')


(333,)

(112,)

/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


auc: 0.78
Accuracy: 76.2763%
auc: 0.72
Accuracy: 68.7500%
success


# LiftChart

In [36]:
## data_scored
data_scored_train['sample_set'] = "train"
data_scored_test['sample_set'] = "test"
data_scored_all = pd.concat([data_scored_train, data_scored_test])
data_scored_all['order_no'] = data_scored_all['order_no'].astype(str)
data_scored_all.head()
save_data_to_pickle(data_scored_all,result_path,'data_scored_all.pkl')

FIG_PATH = os.path.join(result_path, 'figure', 'liftchart')
if not os.path.exists(FIG_PATH):
    os.makedirs(FIG_PATH)

train_lc = pt.show_result_new(data_scored_all.loc[data_scored_all.sample_set == 'train'], 'y_pred','Y', n_bins = 5, feature_label='train')
test_lc = pt.show_result_new(data_scored_all.loc[data_scored_all.sample_set == 'test'], 'y_pred','Y', n_bins = 5, feature_label='test')
# oot_lc = pt.show_result_new(data_scored_all.loc[data_scored_all.sample_set == 'oot'], 'y_pred','Y', n_bins = 10, feature_label='oot')
path = os.path.join(FIG_PATH, "LiftChart.png")
plt.savefig(path, format='png', dpi=300, bbox_inches = 'tight',pad_inches = 0.1)
plt.show()
plt.close()

print('end')

,order_no,Y,y_pred,score,score_bin,score_bin_20,proba_bin,proba_bin_20,sample_set
0,3808899799,0.0,0.608519,587.0,"(513.0, 599.0]","(585.0, 599.0]","(0.505073, 0.951835]","(0.505073, 0.629982]",train
1,3808916175,1.0,0.251753,631.0,"(617.0, 635.0]","(627.0, 635.0]","(0.226033, 0.353034]","(0.226033, 0.28132]",train
2,3808778599,0.0,0.158780,648.0,"(635.0, 661.0]","(644.0, 661.0]","(0.107365, 0.226033]","(0.107365, 0.178454]",train
3,3808862384,0.0,0.074240,673.0,"(661.0, 746.0]","(661.0, 678.0]","(0.0062350899999999996, 0.107365]","(0.0624472, 0.107365]",train
4,3808855137,0.0,0.368454,616.0,"(599.0, 617.0]","(609.0, 617.0]","(0.353034, 0.505073]","(0.353034, 0.420098]",train


分箱数量:  5
分箱数量:  5


/Users/yantingting/PycharmProjects/modeling/plot_tools.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bkl_%s' % y_pred] = pd.qcut(df[y_pred], n_bins, duplicates='drop')
/Users/yantingting/PycharmProjects/modeling/plot_tools.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bkl_%s' % y_pred] = pd.qcut(df[y_pred], n_bins, duplicates='drop')
/Users/yantingting/PycharmProjects/modeling/plot_tools.py:86: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes cur

end


# PDP

In [37]:
FIG_PATH = os.path.join(result_path, 'figure', 'PDP')
if not os.path.exists(FIG_PATH):
    os.makedirs(FIG_PATH)

f_imp_list = LR_final.params.index.tolist()
select_features =LR_final.params.index.tolist()
all_pred = pd.concat([X_transformed_train, X_transformed_test])
len(select_features)    
    
n = 0
while n < len(f_imp_list):
    m = n + 9
    features_draw = [i for i in f_imp_list[n:m]]
    pt.pdpCharts9(LR_final, all_pred, features_draw, select_features, n_bins=10, dfltValue=-1)
    path = os.path.join(FIG_PATH, "pdp_" + str(n) + "_" + str(m) + ".png")
    plt.savefig(path, format='png', dpi=300, bbox_inches = 'tight',pad_inches = 0.1);
    plt.close()
    n += 9
print('end')

18

[<Figure size 1600x800 with 9 Axes>]

[<Figure size 1600x800 with 9 Axes>]

end


# 入模变量的univariate_chart

In [38]:
result_var_path= os.path.join(result_path,'var_used')
if not os.path.exists(result_var_path):
    os.mkdir(result_var_path)
    
time1 = time.time()
feature_list = feature_used
wrong_list= []
for index,value in enumerate(feature_list):
    print(index,value)
    try:
        pt.univariate_chart(df = train_df.copy(), col = value , target ='label' , n = 5,
                             special_attribute = [-1, np.nan],
                             dftrain=train_df.copy(), dftest=test_df.copy(),
                             draw_all=False, draw_train_test=True,result_path = result_var_path)
    except Exception as e:
        wrong_list.append(value)
        print(e)
wrong_list
time2 = time.time()
print('run_time: ',time2 - time1)

0 screen_(2016, 1080)


/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1 state_name_BA
2 email1_outlook
3 boa_vista_score


/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


4 active_companies
5 cnt_len_is9


...

15 model_Moto G (5S) Plus


/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


16 bank_number_0104


[]

run_time:  18.719236850738525
